## This notebook is intended to work on models that needed higher precesion, accuracy, for explainable model, refer to telcomchurnExplainablemodels.ipynb

In [140]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/telecom-churn-case-study-hackathon-c40/data_dictionary.csv
/kaggle/input/telecom-churn-case-study-hackathon-c40/train.csv
/kaggle/input/telecom-churn-case-study-hackathon-c40/test.csv
/kaggle/input/telecom-churn-case-study-hackathon-c40/solution.csv


In [141]:
#pd.set_option('display.max_rows', 30)

In [142]:
telcom_data = pd.read_csv('/kaggle/input/telecom-churn-case-study-hackathon-c40/train.csv');
telcom_data.set_index('id', inplace=True)

In [143]:
telcom_data.head()

,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,arpu_6,arpu_7,arpu_8,...,sachet_3g_7,sachet_3g_8,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,churn_probability
id,,,,,,,,,,,,,,,,,,,,,
0,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,31.277,87.009,7.527,...,0,0,NaN,NaN,NaN,1958,0.0,0.0,0.0,0
1,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,0.000,122.787,42.953,...,0,0,NaN,1.0,NaN,710,0.0,0.0,0.0,0
2,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,60.806,103.176,0.000,...,0,0,NaN,NaN,NaN,882,0.0,0.0,0.0,0
3,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,156.362,205.260,111.095,...,0,0,NaN,NaN,NaN,982,0.0,0.0,0.0,0
4,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,240.708,128.191,101.565,...,1,0,1.0,1.0,1.0,647,0.0,0.0,0.0,0


In [144]:
telcom_data.shape

(69999, 171)

In [145]:
pd.set_option('display.max_rows', None)
telcom_data.dtypes
#pd.set_option('display.max_rows', 30)

circle_id                     int64
loc_og_t2o_mou              float64
std_og_t2o_mou              float64
loc_ic_t2o_mou              float64
last_date_of_month_6         object
last_date_of_month_7         object
last_date_of_month_8         object
arpu_6                      float64
arpu_7                      float64
arpu_8                      float64
onnet_mou_6                 float64
onnet_mou_7                 float64
onnet_mou_8                 float64
offnet_mou_6                float64
offnet_mou_7                float64
offnet_mou_8                float64
roam_ic_mou_6               float64
roam_ic_mou_7               float64
roam_ic_mou_8               float64
roam_og_mou_6               float64
roam_og_mou_7               float64
roam_og_mou_8               float64
loc_og_t2t_mou_6            float64
loc_og_t2t_mou_7            float64
loc_og_t2t_mou_8            float64
loc_og_t2m_mou_6            float64
loc_og_t2m_mou_7            float64
loc_og_t2m_mou_8            

In [146]:
# clean the data for safe checks like empty columns and empty rows...
def sanity_clean_data(telcom_data):
    #columns with all empty values to be removed
    telcom_data.dropna(axis=1, how='all', inplace=True)
    
    #columns with all same values to be removed as they dont provide any insight
    cols_nounique = [i for i in telcom_data.columns if telcom_data[i].nunique()==1]
    telcom_data.drop(columns=cols_nounique, inplace=True, axis=1)
    print(cols_nounique)
    
    #rows with all NAs to be removed
    telcom_data.dropna(axis=0,how='all',inplace=True)
    
sanity_clean_data(telcom_data)    

['circle_id', 'loc_og_t2o_mou', 'std_og_t2o_mou', 'loc_ic_t2o_mou', 'last_date_of_month_6', 'last_date_of_month_7', 'last_date_of_month_8', 'std_og_t2c_mou_6', 'std_og_t2c_mou_7', 'std_og_t2c_mou_8', 'std_ic_t2o_mou_6', 'std_ic_t2o_mou_7', 'std_ic_t2o_mou_8']


In [147]:
telcom_data.shape

(69999, 158)

In [148]:
#since the data is only from year 2014, converting it to day of the year.
# to keep it extensible for any other years, multiplying it with days in year + day in year
def reform_dates(telcom_data):
    date_cols = ['date_of_last_rech_6', 'date_of_last_rech_7', 'date_of_last_rech_8', 'date_of_last_rech_data_6', 'date_of_last_rech_data_7', 'date_of_last_rech_data_8']
    for col_name in date_cols:
        telcom_data[col_name] = pd.to_datetime(telcom_data[col_name]).dt.year*366 + pd.to_datetime(telcom_data[col_name]).dt.dayofyear
    return telcom_data
telcom_data = reform_dates(telcom_data)

In [149]:
telcom_data.columns[telcom_data.isna().any()]

Index(['onnet_mou_6', 'onnet_mou_7', 'onnet_mou_8', 'offnet_mou_6',
       'offnet_mou_7', 'offnet_mou_8', 'roam_ic_mou_6', 'roam_ic_mou_7',
       'roam_ic_mou_8', 'roam_og_mou_6',
       ...
       'arpu_3g_8', 'arpu_2g_6', 'arpu_2g_7', 'arpu_2g_8', 'night_pck_user_6',
       'night_pck_user_7', 'night_pck_user_8', 'fb_user_6', 'fb_user_7',
       'fb_user_8'],
      dtype='object', length=114)

In [150]:
telcom_data['fb_user_6'].unique()

array([nan,  1.,  0.])

In [151]:
telcom_data.columns[telcom_data.isnull().mean() > 0.70]

Index(['date_of_last_rech_data_6', 'date_of_last_rech_data_7',
       'date_of_last_rech_data_8', 'total_rech_data_6', 'total_rech_data_7',
       'total_rech_data_8', 'max_rech_data_6', 'max_rech_data_7',
       'max_rech_data_8', 'count_rech_2g_6', 'count_rech_2g_7',
       'count_rech_2g_8', 'count_rech_3g_6', 'count_rech_3g_7',
       'count_rech_3g_8', 'av_rech_amt_data_6', 'av_rech_amt_data_7',
       'av_rech_amt_data_8', 'arpu_3g_6', 'arpu_3g_7', 'arpu_3g_8',
       'arpu_2g_6', 'arpu_2g_7', 'arpu_2g_8', 'night_pck_user_6',
       'night_pck_user_7', 'night_pck_user_8', 'fb_user_6', 'fb_user_7',
       'fb_user_8'],
      dtype='object')

There are columns that are having more than 70% missing values.
but looking at these columns, it completely makes sense as they are not applicable. hence we need to impute them appropriately

Here, date of last recharge can be imputed with mode and other columns with zero as it is not applicable

In [152]:
#Lets look at the data dictionary and classify numeric vs categorical columns
cat_col_prefix = ['date_of_last_rech_','date_of_last_rech_data_']
cat_cols = []
for i in range(6,9):
    for prefix in cat_col_prefix:
        cat_cols.append(prefix+str(i));
cat_cols

['date_of_last_rech_6',
 'date_of_last_rech_data_6',
 'date_of_last_rech_7',
 'date_of_last_rech_data_7',
 'date_of_last_rech_8',
 'date_of_last_rech_data_8']

In [153]:
num_cols = [i for i in telcom_data.columns if i not in cat_cols]

In [154]:
non_zero_cols = ['aon']

In [155]:
#Missing values imputation:
# impute all numeric cols with median and categorical ones with mode
#for col in num_cols:
#    telcom_data[col].fillna(telcom_data[col].median(), inplace=True)
def impute_missing_values(data):
    #impute non applicable data with zeros
    data[non_zero_cols] = data.filter(non_zero_cols).fillna(data.median(numeric_only=True).iloc[0])
    data[num_cols] = data.filter(num_cols).fillna(0)
    data[cat_cols] = data.filter(cat_cols).fillna(data.mode().iloc[0])
    return data;
telcom_data = impute_missing_values(telcom_data)

In [156]:
telcom_data.shape

(69999, 158)

In [157]:
X = telcom_data.iloc[:,0:telcom_data.shape[1]-1]
y = telcom_data['churn_probability']

In [ ]:
print('Before OverSampling, the shape of train_X: {}'.format(X_train.shape))
print('Before OverSampling, the shape of train_y: {} \n'.format(y_train.ravel().shape))
print("Before OverSampling, counts of label '1': {}".format(sum(y_train == 1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train == 0)))

# import SMOTE module from imblearn library
# pip install imblearn (if you don't have imblearn in your system)
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 2)
X, y = sm.fit_resample(X, y.ravel())

print('After OverSampling, the shape of train_X: {}'.format(X_train.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_train == 1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train == 0)))


In [158]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
#scale the data
scaler = StandardScaler()
X_std = scaler.fit_transform(X)

#PCA
pca = PCA(n_components=0.95)
X_train_pca = pca.fit_transform(X_std)
X_train_pca.shape

(69999, 77)

In [159]:
## Use Random forest classifier

In [160]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV 
classifier_rf = RandomForestClassifier(random_state=42, n_jobs=-1)
# Create the parameter grid based on the results of random search 
params = {
    'max_depth': [4, 6, 10],
    'min_samples_leaf': [100, 500, 1000, 5000],
    'max_features': [5, 10],
    'n_estimators': [10, 50, 100, 1000],
}
# Instantiate the grid search model
grid_search = GridSearchCV(estimator=classifier_rf, param_grid=params, 
                          cv=4, n_jobs=-1, verbose=2, scoring = 'roc_auc')

In [161]:
%%time
grid_search.fit(X_train_pca,y)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.34 µs


In [162]:
rf_best = grid_search.best_estimator_
rf_best

In [163]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y, rf_best.predict(X_train_pca))

In [164]:
accuracy_score(y,rf_best.predict(X_train_pca))

In [165]:
from sklearn.metrics import precision_score
precision_score(y, rf_best.predict(X_train_pca), average=None)

In [166]:

test_data = pd.read_csv('/kaggle/input/telecom-churn-case-study-hackathon-c40/test.csv', header=0);
#test_data.set_index('id', inplace=True)
test_data = reform_dates(test_data)
test_data['churn_probability'] = 0
test_data = impute_missing_values(test_data)
y_test = test_data.pop('churn_probability')
X_test = test_data[X.columns]
X_test_std = scaler.transform(X_test)

X_test_pca = pca.transform(X_test_std)
test_data['churn_probability'] = rf_best.predict(X_test_pca)
solution = test_data[['id', 'churn_probability']]
solution.set_index('id', inplace=True)
solution.to_csv('solution.csv', header=True, index=True)

"\n\ntest_data = pd.read_csv('/kaggle/input/telecom-churn-case-study-hackathon-c40/test.csv', header=0);\n#test_data.set_index('id', inplace=True)\ntest_data = reform_dates(test_data)\ntest_data['churn_probability'] = 0\ntest_data = impute_missing_values(test_data)\ny_test = test_data.pop('churn_probability')\nX_test = test_data[X.columns]\nX_test_std = scaler.transform(X_test)\n\nX_test_pca = pca.transform(X_test_std)\ntest_data['churn_probability'] = rf_best.predict(X_test_pca)\nsolution = test_data[['id', 'churn_probability']]\nsolution.set_index('id', inplace=True)\nsolution.to_csv('solution.csv', header=True, index=True)\n"

##Using Boosting

In [171]:

import xgboost as xgb  # Load this xgboost
xgb_cfl = xgb.XGBClassifier(n_jobs = -1,objective = 'binary:logistic')
xgb_cfl.get_params()
# Fit the model to our train and target
xgb_cfl.fit(X_train_pca, y)  # default 

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=-1, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [172]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y, xgb_cfl.predict(X_train_pca))

array([[62502,   365],
       [ 1360,  5772]])

In [173]:
X_test_std = scaler.transform(X_test)

In [174]:
X_test_pca = pca.transform(X_test_std)
test_data['churn_probability'] = xgb_cfl.predict(X_test_pca)
solution = test_data[['id', 'churn_probability']]
solution.set_index('id', inplace=True)
solution.to_csv('solution.csv', header=True, index=True)

## This notebook is intended to work on models that needed higher precesion, accuracy, for explainable model, refer to telcomchurnExplainablemodels.ipynb